In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!pip install transformers==2.5.1

In [10]:
import torch
import torch.nn as nn 
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import RobertaConfig, RobertaModel
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
import tensorflow as tf
# from transformers import AutoTokenizer, BertTokenizer, EvalPrediction, BertPreTrainedModel, BertConfig, BertModel, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

import random
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [11]:
    import torch
    torch.cuda.empty_cache()

In [12]:
cd '/content/drive/MyDrive/'

/content/drive/MyDrive


## Data pre-processing

Strict-matched

In [13]:
circa_og = pd.read_csv('/content/drive/MyDrive/NLU_Project/circa-data.tsv', sep='\t', index_col='id')
# circa_og = pd.read_csv('/content/drive/MyDrive/AMBER_TENG_NYU_DRIVE_TO_BROWN/NYU_SPRING2021/NLU/NLU_Project/circa-data.tsv', sep='\t', index_col='id')
# circa_r = circa_og.drop(circa_og.loc[circa_og['goldstandard2']=='Other'].index)
circa_r = circa_og.drop(circa_og.loc[circa_og['goldstandard2'].isnull()].index)

In [14]:
# circa_og = pd.read_csv('circa-data.tsv', sep='\t', index_col='id')
# circa_og = pd.read_csv('/content/drive/MyDrive/NLU_Project/circa-data.tsv', sep='\t', index_col='id')
circa_s = circa_og.drop(circa_og.loc[circa_og['goldstandard1']=='Other'].index)
circa_s = circa_s.drop(circa_s.loc[circa_s['goldstandard1'].isnull()].index)
circa_s = circa_s.drop(circa_s.loc[circa_s['goldstandard1']=='I am not sure how X will interpret Y’s answer'].index)

In [15]:
# YN_r = (circa_r['question-X'].map(str)+' '+circa_r['answer-Y']).apply(lambda row: row.strip())
# relaxed_labels = circa_r['goldstandard2'].unique()
# relaxed_label = circa_r['goldstandard2']
# relaxed_dict = {}
# for idx, label in enumerate(relaxed_labels):
#     relaxed_dict[label] = idx
# circa_r['relaxed'] = circa_r.goldstandard2.replace(relaxed_dict)
# relaxed = circa_r['relaxed']
YN_s = (circa_s['question-X'].map(str)+' '+circa_s['answer-Y']).apply(lambda row: row.strip())
strict_labels = circa_s['goldstandard1'].unique()
strict_label = circa_s['goldstandard1']
strict_dict = {}
for idx, label in enumerate(strict_labels):
    strict_dict[label] = idx
circa_s['strict'] = circa_s.goldstandard1.replace(strict_dict)
strict = circa_s['strict']

## Modeling

In [16]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()
print(device_name)

if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

/device:GPU:0


### Strict

In [17]:
from transformers import PreTrainedModel
from transformers import BertConfig, BertModel
from transformers.modeling_tf_bert import TFBertForSequenceClassification
from transformers import BertForSequenceClassification

from transformers import RobertaForSequenceClassification

## Model Specs
- BERT-MNLI-YN Relaxed learning rate: 5e-5
- MNLI learning rate: 2e-5 ; 3 epochs; batch size 16 vs 32 
  - https://huggingface.co/ishan/bert-base-uncased-mnli 
  - The training parameters were kept the same as Devlin et al., 2019 (learning rate = 2e-5, training epochs = 3, max_sequence_len = 128 and batch_size = 32).
- BERT-MNLI-YN Strict learning rate: 2e-5

In [18]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base') 
# tokenizer = AutoTokenizer.from_pretrained('ishan/bert-base-uncased-mnli') 
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)
# device = torch.device("cpu")
# Tell pytorch to run this model on the GPU.
# model.cuda()
model.to(device)

learning_rate = 2e-5
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=1e-8)

In [19]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  490476 KB |  490476 KB |  490476 KB |       0 B  |\n|       from large pool |  489984 KB |  489984 KB |  489984 KB |       0 B  |\n|       from small pool |     492 KB |     492 KB |     492 KB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |  490476 KB |  490476 KB |  490476 KB |       0 B  |\n|       from large pool |  489984 KB |  489984 KB |

In [20]:
# max_len = 0
# for entry in YN_r.values:
#     input_ids = tokenizer.encode(entry,  add_special_tokens=True)
#     max_len = max(max_len, len(input_ids))
# print(max_len)

max_len = 0
for entry in YN_s.values:
    input_ids = tokenizer.encode(entry,  add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
print(max_len)

41


In [21]:
strict_labels = circa_s['goldstandard1'].unique()
strict_label = circa_s['goldstandard1']
strict_dict = {}
for idx, label in enumerate(strict_labels):
    strict_dict[label] = idx
circa_s['strict'] = circa_s.goldstandard1.replace(strict_dict)
strict = circa_s['strict']

YN_s = (circa_s['question-X'].map(str)+' '+circa_s['answer-Y']).apply(lambda row: row.strip())

In [22]:
# df = pd.concat([YN_r, relaxed_label, relaxed], axis=1).rename(columns={0:'YN_r'})
# df
df = pd.concat([YN_s, strict_label, strict], axis=1).rename(columns={0:'YN_s'})
df

,YN_s,goldstandard1,strict
id,,,
0,Are you employed? I'm a veterinary technician.,Yes,0
1,Are you a fan of Korean food? I wouldn't say so,No,1
2,Are you bringing any pets into the flat? I do ...,No,1
3,Would you like to get some fresh air in your f...,Yes,0
4,Is your family still living in the neighborhoo...,"In the middle, neither yes nor no",2
...,...,...,...
34263,Do you like to drink? I am in AA.,No,1
34264,Do you like pie? My favorite pie is pecan.,Yes,0
34265,Want to go to a concert with me? I'd rather do...,No,1


In [23]:
# train_relaxed, val_relaxed, trainy_relaxed, valy_relaxed = train_test_split(df.index.values, df.relaxed.values, test_size=.4, stratify=df.relaxed.values)
# test_relaxed, dev_relaxed, testy_relaxed, devy_relaxed = train_test_split(val_relaxed, valy_relaxed, test_size=.5, stratify=valy_relaxed)

train_strict, val_strict, trainy_strict, valy_strict = train_test_split(df.index.values, df.strict.values, test_size=.4, stratify=df.strict.values)
test_strict, dev_strict, testy_strict, devy_strict = train_test_split(val_strict, valy_strict, test_size=.5, stratify=valy_strict)

In [24]:
# df['data_type'] = ['not_set']*df.shape[0]
# df.loc[train_relaxed,'data_type'] = 'train'
# df.loc[dev_relaxed,'data_type'] = 'dev'
# df.loc[test_relaxed,'data_type'] = 'test'

df['data_type'] = ['not_set']*df.shape[0]
df.loc[train_strict,'data_type'] = 'train'
df.loc[dev_strict,'data_type'] = 'dev'
df.loc[test_strict,'data_type'] = 'test'

In [25]:
# df.groupby(['goldstandard2','relaxed','data_type']).count()

df.groupby(['goldstandard1','strict','data_type']).count()

YN_s
goldstandard1                     strict data_type      
In the middle, neither yes nor no 2      dev         127
                                         test        128
                                         train       383
No                                1      dev        2166
                                         test       2166
                                         train      6497
Probably no                       4      dev         232
                                         test        232
                                         train       696
Probably yes / sometimes yes      3      dev         249
                                         test        249
                                         train       746
Yes                               0      dev        2901
                                         test       2901
                                         train      8702
Yes, subject to some conditions   5      dev         517
                                         test        516
                                         train      1550

In [26]:
# encoded_data_train = tokenizer.batch_encode_plus(
#     df[df.data_type=='train'].YN_r.values, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     pad_to_max_length=True, 
#     max_length=256, 
#     return_tensors='pt'
# )

# encoded_data_dev = tokenizer.batch_encode_plus(
#     df[df.data_type=='dev'].YN_r.values, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     pad_to_max_length=True, 
#     max_length=256, 
#     return_tensors='pt'
# )


# input_ids_train = encoded_data_train['input_ids']
# attention_masks_train = encoded_data_train['attention_mask']
# labels_train = torch.tensor(df[df.data_type=='train'].relaxed.values)

# input_ids_dev = encoded_data_dev['input_ids']
# attention_masks_dev = encoded_data_dev['attention_mask']
# labels_dev = torch.tensor(df[df.data_type=='dev'].relaxed.values)

encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].YN_s.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_dev = tokenizer.batch_encode_plus(
    df[df.data_type=='dev'].YN_s.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].strict.values)

input_ids_dev = encoded_data_dev['input_ids']
attention_masks_dev = encoded_data_dev['attention_mask']
labels_dev = torch.tensor(df[df.data_type=='dev'].strict.values)

In [27]:
# encoded_data_train
# input_ids_train
# attention_masks_dev
# labels_dev  

In [28]:
# dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
# dataset_dev = TensorDataset(input_ids_dev, attention_masks_dev, labels_dev)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_dev = TensorDataset(input_ids_dev, attention_masks_dev, labels_dev)

In [29]:
len(dataset_train)

18574

In [30]:
# model
# relaxed_dict
len(strict_dict)

6

In [31]:
model.classifier

RobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=3, bias=True)
)

In [32]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [33]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base',
                                                      num_labels=len(strict_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

# model = BertForSequenceClassification.from_pretrained('ishan/bert-base-uncased-mnli',
#                                                       num_labels=4,
#                                                       # num_labels=len(relaxed_dict),
#                                                       output_attentions=False,
#                                                       output_hidden_states=False)
# model.classifier = torch.nn.Linear(model.classifier.in_features, 6)
model.num_labels = 6
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=1e-8)
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [34]:
batch_size = 32
# batch_size = 16

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_dev, 
                                   sampler=SequentialSampler(dataset_dev), 
                                   batch_size=batch_size)

optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = 1e-8)

In [35]:
epochs = 3
# epochs = 2
total_steps = len(dataloader_train) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [41]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average=None)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in strict_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [42]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [43]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
model.to(device)

print(device)

cuda


In [44]:

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [45]:

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    dev_loss, predictions, true_vals = evaluate(dataloader_validation)
    dev_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {dev_loss}')
    tqdm.write(f'F1 Score (Weighted): {dev_f1}')


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/581 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/581 [00:00<?, ?it/s, training_loss=0.100]

Epoch 1:   0%|          | 1/581 [00:00<07:50,  1.23it/s, training_loss=0.100]

Epoch 1:   0%|          | 1/581 [00:01<07:50,  1.23it/s, training_loss=0.222]

Epoch 1:   0%|          | 2/581 [00:01<07:45,  1.24it/s, training_loss=0.222]

Epoch 1:   0%|          | 2/581 [00:02<07:45,  1.24it/s, training_loss=0.151]

Epoch 1:   1%|          | 3/581 [00:02<07:40,  1.25it/s, training_loss=0.151]

Epoch 1:   1%|          | 3/581 [00:03<07:40,  1.25it/s, training_loss=0.242]

Epoch 1:   1%|          | 4/581 [00:03<07:38,  1.26it/s, training_loss=0.242]

Epoch 1:   1%|          | 4/581 [00:03<07:38,  1.26it/s, training_loss=0.183]

Epoch 1:   1%|          | 5/581 [00:03<07:36,  1.26it/s, training_loss=0.183]

Epoch 1:   1%|          | 5/581 [00:04<07:36,  1.26it/s, training_loss=0.248]

Epoch 1:   1%|          | 6/581 [00:04<07:34,  1.


Epoch 1
Training loss: 0.3084158421383462



  0%|          | 0/3 [23:49<?, ?it/s]

 33%|███▎      | 1/3 [08:29<16:59, 509.52s/it]

Epoch 2:   0%|          | 0/581 [00:00<?, ?it/s]

Validation loss: 0.48380586505896345
F1 Score (Weighted): [0.91717172 0.88595115 0.39583333 0.54117647 0.29325513 0.89540567]




Epoch 2:   0%|          | 0/581 [00:00<?, ?it/s, training_loss=0.060]

Epoch 2:   0%|          | 1/581 [00:00<07:39,  1.26it/s, training_loss=0.060]

Epoch 2:   0%|          | 1/581 [00:01<07:39,  1.26it/s, training_loss=0.057]

Epoch 2:   0%|          | 2/581 [00:01<07:40,  1.26it/s, training_loss=0.057]

Epoch 2:   0%|          | 2/581 [00:02<07:40,  1.26it/s, training_loss=0.028]

Epoch 2:   1%|          | 3/581 [00:02<07:38,  1.26it/s, training_loss=0.028]

Epoch 2:   1%|          | 3/581 [00:03<07:38,  1.26it/s, training_loss=0.169]

Epoch 2:   1%|          | 4/581 [00:03<07:37,  1.26it/s, training_loss=0.169]

Epoch 2:   1%|          | 4/581 [00:03<07:37,  1.26it/s, training_loss=0.045]

Epoch 2:   1%|          | 5/581 [00:03<07:38,  1.26it/s, training_loss=0.045]

Epoch 2:   1%|          | 5/581 [00:04<07:38,  1.26it/s, training_loss=0.054]

Epoch 2:   1%|          | 6/581 [00:04<07:37,  1.26it/s, training_loss=0.054]

Epoch 2:   1%|          | 6/581 [00:05<07:37,  1.26it/s, t

...

Validation loss: 0.4412561918695256
F1 Score (Weighted): [0.92140553 0.89167975 0.52293578 0.54368932 0.34444444 0.90038314]




Epoch 3:   0%|          | 0/581 [00:00<?, ?it/s, training_loss=0.075]

Epoch 3:   0%|          | 1/581 [00:00<07:44,  1.25it/s, training_loss=0.075]

Epoch 3:   0%|          | 1/581 [00:01<07:44,  1.25it/s, training_loss=0.089]

Epoch 3:   0%|          | 2/581 [00:01<07:42,  1.25it/s, training_loss=0.089]

Epoch 3:   0%|          | 2/581 [00:02<07:42,  1.25it/s, training_loss=0.141]

Epoch 3:   1%|          | 3/581 [00:02<07:40,  1.26it/s, training_loss=0.141]

Epoch 3:   1%|          | 3/581 [00:03<07:40,  1.26it/s, training_loss=0.071]

Epoch 3:   1%|          | 4/581 [00:03<07:37,  1.26it/s, training_loss=0.071]

Epoch 3:   1%|          | 4/581 [00:03<07:37,  1.26it/s, training_loss=0.182]

Epoch 3:   1%|          | 5/581 [00:03<07:35,  1.26it/s, training_loss=0.182]

Epoch 3:   1%|          | 5/581 [00:04<07:35,  1.26it/s, training_loss=0.094]

Epoch 3:   1%|          | 6/581 [00:04<07:34,  1.26it/s, training_loss=0.094]

Epoch 3:   1%|          | 6/581 [00:05<07:34,  1.26it/s, t


Epoch 3
Training loss: 0.25852903510843006



  0%|          | 0/3 [40:58<?, ?it/s]

100%|██████████| 3/3 [25:37<00:00, 512.65s/it]

Validation loss: 0.4412561918695256
F1 Score (Weighted): [0.92140553 0.89167975 0.52293578 0.54368932 0.34444444 0.90038314]


In [46]:
accuracy_per_class(predictions, true_vals)

Class: Yes
Accuracy: 2714/2901

Class: No
Accuracy: 1988/2166

Class: In the middle, neither yes nor no
Accuracy: 57/127

Class: Probably yes / sometimes yes
Accuracy: 112/249

Class: Probably no
Accuracy: 62/232

Class: Yes, subject to some conditions
Accuracy: 470/517



In [47]:
print('Dev Accuracy:', end = ' ')
flat_accuracy(predictions, true_vals)

Dev Accuracy: 

0.872577519379845

In [48]:
encoded_data_test = tokenizer.batch_encode_plus(
    df[df.data_type=='test'].YN_s.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df[df.data_type=='test'].strict.values)

dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

test_loss, test_predictions, test_true_vals = evaluate(dataloader_test)
test_f1 = f1_score_func(test_predictions, test_true_vals)

In [49]:
accuracy_per_class(test_predictions, test_true_vals)
print('Test Accuracy:', end = ' ')
flat_accuracy(test_predictions, test_true_vals)

Class: Yes
Accuracy: 2699/2901

Class: No
Accuracy: 1980/2166

Class: In the middle, neither yes nor no
Accuracy: 58/128

Class: Probably yes / sometimes yes
Accuracy: 92/249

Class: Probably no
Accuracy: 53/232

Class: Yes, subject to some conditions
Accuracy: 473/516

Test Accuracy: 

0.8648255813953488

In [50]:
test_f1


array([0.91507035, 0.88392857, 0.50434783, 0.47179487, 0.30994152,
       0.90699904])

In [51]:
print(test_loss, test_predictions, test_true_vals)

0.45164522012100394 [[ 0.53265506 -3.142577   -0.12642288  4.7017865  -1.5508634  -1.0550734 ]
 [ 0.16815327  6.1785097  -1.0919583  -2.1432967   0.37265533 -1.977092  ]
 [ 5.587841   -2.3524048  -1.0633026   1.9100188  -2.327075   -1.1745166 ]
 ...
 [ 3.6309      1.6267486  -0.92058605  0.49059317 -0.86259145 -3.216081  ]
 [ 4.0780964  -0.6314746  -0.9052694   0.77563447 -1.100635   -1.9468657 ]
 [ 3.765187   -2.0813878  -1.4303197  -0.13429727 -2.1205595   2.335991  ]] [3 1 0 ... 0 3 0]


In [52]:
# groupwise f1 scores 

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='none')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in strict_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [54]:
# list(df.groupby(df[answer]).groups.keys())